In [1]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

In [2]:
spark = SparkSession \
    .builder \
    .appName("PySpark Experimentation") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 10:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_df = spark.createDataFrame([
    (1, "2023-06-01", 5, 4, "option_1"),
    (1, "2023-06-01", 6, 1, "option_2"),
    (1, "2023-06-02", 7, 6, "option_1"),
    (1, "2023-06-06", 4, 2, "option_1"),
    (2, "2023-06-03", 10, 12, "option_2"),
    (2, "2023-06-03", 13, 15, "option_2"),
], schema=["id", "timestamp", "numerical_1", "numerical_2", "categorical_feature"])

data_df = data_df.withColumn("timestamp", F.to_timestamp(F.col("timestamp"), "yyyy-MM-dd"))

In [4]:
data_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- numerical_1: long (nullable = true)
 |-- numerical_2: long (nullable = true)
 |-- categorical_feature: string (nullable = true)



In [5]:
data_df.show()

+---+-------------------+-----------+-----------+-------------------+
| id|          timestamp|numerical_1|numerical_2|categorical_feature|
+---+-------------------+-----------+-----------+-------------------+
|  1|2023-06-01 00:00:00|          5|          4|           option_1|
|  1|2023-06-01 00:00:00|          6|          1|           option_2|
|  1|2023-06-02 00:00:00|          7|          6|           option_1|
|  1|2023-06-06 00:00:00|          4|          2|           option_1|
|  2|2023-06-03 00:00:00|         10|         12|           option_2|
|  2|2023-06-03 00:00:00|         13|         15|           option_2|
+---+-------------------+-----------+-----------+-------------------+



In [6]:
utf_shift_hours = 2
windows_size = "2 days"

In [7]:
window_column = F.window(timeColumn=F.col("timestamp"), windowDuration=windows_size, startTime=f"-{utf_shift_hours} hours")

In [8]:
data_df = data_df.withColumn("window", window_column)

In [9]:
data_df.show(truncate=False)

+---+-------------------+-----------+-----------+-------------------+------------------------------------------+
|id |timestamp          |numerical_1|numerical_2|categorical_feature|window                                    |
+---+-------------------+-----------+-----------+-------------------+------------------------------------------+
|1  |2023-06-01 00:00:00|5          |4          |option_1           |{2023-05-31 00:00:00, 2023-06-02 00:00:00}|
|1  |2023-06-01 00:00:00|6          |1          |option_2           |{2023-05-31 00:00:00, 2023-06-02 00:00:00}|
|1  |2023-06-02 00:00:00|7          |6          |option_1           |{2023-06-02 00:00:00, 2023-06-04 00:00:00}|
|1  |2023-06-06 00:00:00|4          |2          |option_1           |{2023-06-06 00:00:00, 2023-06-08 00:00:00}|
|2  |2023-06-03 00:00:00|10         |12         |option_2           |{2023-06-02 00:00:00, 2023-06-04 00:00:00}|
|2  |2023-06-03 00:00:00|13         |15         |option_2           |{2023-06-02 00:00:00, 2023-

In [10]:
# Solution 1

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature").count()
cat_grouped_df.show()

+---+--------------------+--------+--------+
| id|              window|option_1|option_2|
+---+--------------------+--------+--------+
|  1|{2023-05-31 00:00...|       1|       1|
|  1|{2023-06-06 00:00...|       1|    null|
|  2|{2023-06-02 00:00...|    null|       2|
|  1|{2023-06-02 00:00...|       1|    null|
+---+--------------------+--------+--------+



In [11]:
# Solution 2a

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature").sum("numerical_1", "numerical_2")
cat_grouped_df.show()

+---+--------------------+-------------------------+-------------------------+-------------------------+-------------------------+
| id|              window|option_1_sum(numerical_1)|option_1_sum(numerical_2)|option_2_sum(numerical_1)|option_2_sum(numerical_2)|
+---+--------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|  1|{2023-05-31 00:00...|                        5|                        4|                        6|                        1|
|  1|{2023-06-06 00:00...|                        4|                        2|                     null|                     null|
|  2|{2023-06-02 00:00...|                     null|                     null|                       23|                       27|
|  1|{2023-06-02 00:00...|                        7|                        6|                     null|                     null|
+---+--------------------+-------------------------+-------------------------+-----

In [12]:
# Solution 2b

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature").agg(F.sum("numerical_1").alias(""), F.mean("numerical_2"))
cat_grouped_df.show()

+---+--------------------+---------+-------------------------+---------+-------------------------+
| id|              window|option_1_|option_1_avg(numerical_2)|option_2_|option_2_avg(numerical_2)|
+---+--------------------+---------+-------------------------+---------+-------------------------+
|  1|{2023-05-31 00:00...|        5|                      4.0|        6|                      1.0|
|  1|{2023-06-06 00:00...|        4|                      2.0|     null|                     null|
|  2|{2023-06-02 00:00...|     null|                     null|       23|                     13.5|
|  1|{2023-06-02 00:00...|        7|                      6.0|     null|                     null|
+---+--------------------+---------+-------------------------+---------+-------------------------+



In [13]:
# Solution 2c

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature").agg({"numerical_1": "sum", "numerical_2": "mean"})
cat_grouped_df.show()

+---+--------------------+-------------------------+-------------------------+-------------------------+-------------------------+
| id|              window|option_1_avg(numerical_2)|option_1_sum(numerical_1)|option_2_avg(numerical_2)|option_2_sum(numerical_1)|
+---+--------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|  1|{2023-05-31 00:00...|                      4.0|                        5|                      1.0|                        6|
|  1|{2023-06-06 00:00...|                      2.0|                        4|                     null|                     null|
|  2|{2023-06-02 00:00...|                     null|                     null|                     13.5|                       23|
|  1|{2023-06-02 00:00...|                      6.0|                        7|                     null|                     null|
+---+--------------------+-------------------------+-------------------------+-----

In [14]:
# Solution 2d

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature").agg({"numerical_1": "sum", "numerical_2": "mean"})
cat_grouped_df = cat_grouped_df.select(*(F.col(i).alias(i.replace("(",'_').replace(')','')) for i in cat_grouped_df.columns))
cat_grouped_df.show()

+---+--------------------+------------------------+------------------------+------------------------+------------------------+
| id|              window|option_1_avg_numerical_2|option_1_sum_numerical_1|option_2_avg_numerical_2|option_2_sum_numerical_1|
+---+--------------------+------------------------+------------------------+------------------------+------------------------+
|  1|{2023-05-31 00:00...|                     4.0|                       5|                     1.0|                       6|
|  1|{2023-06-06 00:00...|                     2.0|                       4|                    null|                    null|
|  2|{2023-06-02 00:00...|                    null|                    null|                    13.5|                      23|
|  1|{2023-06-02 00:00...|                     6.0|                       7|                    null|                    null|
+---+--------------------+------------------------+------------------------+------------------------+----------

In [15]:
# Solution 2e

cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature", ["option_1"]).agg({"numerical_1": "sum", "numerical_2": "mean"})
cat_grouped_df.show()

+---+--------------------+-------------------------+-------------------------+
| id|              window|option_1_avg(numerical_2)|option_1_sum(numerical_1)|
+---+--------------------+-------------------------+-------------------------+
|  1|{2023-05-31 00:00...|                      4.0|                        5|
|  1|{2023-06-06 00:00...|                      2.0|                        4|
|  2|{2023-06-02 00:00...|                     null|                     null|
|  1|{2023-06-02 00:00...|                      6.0|                        7|
+---+--------------------+-------------------------+-------------------------+



In [16]:
cat_grouped_df = data_df.groupBy(F.col("id"), window_column).pivot("categorical_feature", ["option_1"]).agg({"numerical_1": "count"})
cat_grouped_df.show()

+---+--------------------+--------+
| id|              window|option_1|
+---+--------------------+--------+
|  1|{2023-05-31 00:00...|       1|
|  1|{2023-06-06 00:00...|       1|
|  2|{2023-06-02 00:00...|    null|
|  1|{2023-06-02 00:00...|       1|
+---+--------------------+--------+



In [17]:
data_df.show()

+---+-------------------+-----------+-----------+-------------------+--------------------+
| id|          timestamp|numerical_1|numerical_2|categorical_feature|              window|
+---+-------------------+-----------+-----------+-------------------+--------------------+
|  1|2023-06-01 00:00:00|          5|          4|           option_1|{2023-05-31 00:00...|
|  1|2023-06-01 00:00:00|          6|          1|           option_2|{2023-05-31 00:00...|
|  1|2023-06-02 00:00:00|          7|          6|           option_1|{2023-06-02 00:00...|
|  1|2023-06-06 00:00:00|          4|          2|           option_1|{2023-06-06 00:00...|
|  2|2023-06-03 00:00:00|         10|         12|           option_2|{2023-06-02 00:00...|
|  2|2023-06-03 00:00:00|         13|         15|           option_2|{2023-06-02 00:00...|
+---+-------------------+-----------+-----------+-------------------+--------------------+



In [18]:
num_grouped_df = data_df.groupBy(F.col("id"), window_column).agg(F.sum("numerical_1").alias("tot"))
num_grouped_df.show()

+---+--------------------+---+
| id|              window|tot|
+---+--------------------+---+
|  1|{2023-05-31 00:00...| 11|
|  1|{2023-06-02 00:00...|  7|
|  1|{2023-06-06 00:00...|  4|
|  2|{2023-06-02 00:00...| 23|
+---+--------------------+---+



In [19]:
grouped_df = cat_grouped_df.join(num_grouped_df, on=["id", "window"])
grouped_df.show()

+---+--------------------+--------+---+
| id|              window|option_1|tot|
+---+--------------------+--------+---+
|  1|{2023-05-31 00:00...|       1| 11|
|  1|{2023-06-02 00:00...|       1|  7|
|  1|{2023-06-06 00:00...|       1|  4|
|  2|{2023-06-02 00:00...|    null| 23|
+---+--------------------+--------+---+



In [20]:
grouped_df = grouped_df.withColumn("timestamp", F.col("window").start)
# grouped_df = grouped_df.withColumn("window_end", F.col("window").end)

grouped_df.show()

+---+--------------------+--------+---+-------------------+
| id|              window|option_1|tot|          timestamp|
+---+--------------------+--------+---+-------------------+
|  1|{2023-05-31 00:00...|       1| 11|2023-05-31 00:00:00|
|  1|{2023-06-02 00:00...|       1|  7|2023-06-02 00:00:00|
|  1|{2023-06-06 00:00...|       1|  4|2023-06-06 00:00:00|
|  2|{2023-06-02 00:00...|    null| 23|2023-06-02 00:00:00|
+---+--------------------+--------+---+-------------------+



In [21]:
ids_df = data_df.select(F.col("id"), F.col("window").start.alias("window_start"), F.col("window").end.alias("window_end")).distinct()
ids_df = ids_df.groupBy("id").agg(F.min("window_start").alias("min_window_start"), F.max("window_end").alias("max_window_end"))
ids_df.show()

+---+-------------------+-------------------+
| id|   min_window_start|     max_window_end|
+---+-------------------+-------------------+
|  1|2023-05-31 00:00:00|2023-06-08 00:00:00|
|  2|2023-06-02 00:00:00|2023-06-04 00:00:00|
+---+-------------------+-------------------+



In [22]:
ids_timestamps_df = ids_df.withColumn("timestamps", F.expr(f"sequence(to_timestamp(min_window_start), to_timestamp(max_window_end), interval {windows_size})")).drop("min_window_start", "max_window_end")
ids_timestamps_df = ids_timestamps_df.withColumn("timestamp", F.explode(F.col("timestamps"))).drop("window_start", "window_end", "timestamps")
ids_timestamps_df.show()

+---+-------------------+
| id|          timestamp|
+---+-------------------+
|  1|2023-05-31 00:00:00|
|  1|2023-06-02 00:00:00|
|  1|2023-06-04 00:00:00|
|  1|2023-06-06 00:00:00|
|  1|2023-06-08 00:00:00|
|  2|2023-06-02 00:00:00|
|  2|2023-06-04 00:00:00|
+---+-------------------+



In [23]:
data_df.show()

+---+-------------------+-----------+-----------+-------------------+--------------------+
| id|          timestamp|numerical_1|numerical_2|categorical_feature|              window|
+---+-------------------+-----------+-----------+-------------------+--------------------+
|  1|2023-06-01 00:00:00|          5|          4|           option_1|{2023-05-31 00:00...|
|  1|2023-06-01 00:00:00|          6|          1|           option_2|{2023-05-31 00:00...|
|  1|2023-06-02 00:00:00|          7|          6|           option_1|{2023-06-02 00:00...|
|  1|2023-06-06 00:00:00|          4|          2|           option_1|{2023-06-06 00:00...|
|  2|2023-06-03 00:00:00|         10|         12|           option_2|{2023-06-02 00:00...|
|  2|2023-06-03 00:00:00|         13|         15|           option_2|{2023-06-02 00:00...|
+---+-------------------+-----------+-----------+-------------------+--------------------+



In [24]:
group_on = (grouped_df.timestamp == ids_timestamps_df.timestamp) & (grouped_df.id == ids_timestamps_df.id)
grouped_df.join(ids_timestamps_df, on=["id", "timestamp"], how='right').fillna(0, subset=["tot", "option_1", "option_2"]).drop("window").show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `option_2` cannot be resolved. Did you mean one of the following? [`id`, `timestamp`, `window`, `option_1`, `tot`].

23/07/13 10:05:37 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/07/13 19:34:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 955957 ms exceeds timeout 120000 ms
23/07/13 19:34:56 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/13 19:35:00 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMa